In [1]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import time
import random
import sqlite3

/Users/kbeisenb/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
driver = webdriver.Firefox()
close_button_class = "tutorial__close notes-tutorial__close"
url = "https://krisha.kz/arenda/kvartiry/almaty/?rent-period-switch=%2Farenda%2Fkvartiry"
driver.get(url)

There was an error managing geckodriver (error sending request for url (https://product-details.mozilla.org/1.0/firefox_versions.json): error trying to connect: dns error: Этот хост неизвестен. (os error 11001)); using driver found in the cache
There was an error managing firefox; using browser found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event): operation timed out


In [2]:
def get_prices(class_name, driver):
    """
    this function finds all prices on krisha.kz
    class_name variable for kirsha.kz = a-card__price
    """
    prices = []
    price = driver.find_elements(By.CLASS_NAME, class_name)
    pattern = r"\b(\d+)\b"
    for i in price:
        matches = re.findall(pattern, i.get_attribute("outerHTML"))
        numeric_value = "".join(matches)
        prices.append(int(numeric_value))
    return prices

In [3]:
def get_titles(class_name, driver):
    """
    this function finds all titles on krisha.kz
    class_name variable for kirsha.kz = a-card__title
    """
    titles = []
    title = driver.find_elements(By.CLASS_NAME, class_name)
    for i in title:
        soup = BeautifulSoup(i.get_attribute("outerHTML"), "html.parser")
        text = soup.get_text()
        titles.append(text.split(", "))
    return titles

In [4]:
def get_subtitles(class_name, driver):
    """
    this function finds all subtitles on krisha.kz
    class_name variable for kirsha.kz = a-card__subtitle
    """
    subtitles = []
    subtitle = driver.find_elements(By.CLASS_NAME, class_name)
    for i in subtitle:
        text = BeautifulSoup(i.get_attribute("outerHTML"), "html.parser").get_text()
        cleaned_text = re.sub(r"\s+", " ", text).strip()
        subtitles.append(cleaned_text)
    return subtitles

In [5]:
def get_owners(class_name, driver):
    """
    this function finds all owners types(owner, company, specialist) on krisha.kz
    class_name variable for kirsha.kz = a-card__owner
    """
    owners_list = []
    owners_raw = driver.find_elements(By.CLASS_NAME, class_name)
    for owners in owners_raw:
        soup = BeautifulSoup(owners.get_attribute("outerHTML"), "html.parser")
        all_owners = soup.find_all("div", class_=class_name)
        for owner in all_owners:
            label = owner.find("div", class_=re.compile(r"label-user-.*")).get_text(strip=True)
            owners_list.append(label)
    return owners_list

In [6]:
def get_text_previews(class_name, driver):
    """
    this function finds all text-previews on krisha.kz
    class_name variable for kirsha.kz = a-card__text-preview
    """
    text = driver.find_elements(By.CLASS_NAME, class_name)
    texts = []
    for i in text:
        soup = BeautifulSoup(i.get_attribute("outerHTML"), "html.parser")
        text_preview = soup.find("div", class_=class_name).get_text(strip=True)
        texts.append(text_preview)
    return texts

In [7]:
def get_urls(driver):
    urls = []
    title = driver.find_elements(By.CLASS_NAME, "a-card__title")
    for i in title:
        soup = BeautifulSoup(i.get_attribute('outerHTML'), 'html.parser')
        href_value = soup.find('a')['href']
        urls.append("https://krisha.kz" + href_value)
    return urls

In [9]:
titles = get_titles("a-card__title", driver)
prices = get_prices("a-card__price", driver)
subtitles = get_subtitles("a-card__subtitle", driver)
owners = get_owners("a-card__owner", driver)
text_previews = get_text_previews("a-card__text-preview", driver)
urls = get_urls(driver)

In [8]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_contact(driver):
    contact = driver.find_element(By.CLASS_NAME, "show-phones")
    contact.click()
    time.sleep(random.randint(1, 2))
    number = driver.find_element(By.CLASS_NAME, "offer__contacts-phones")
    phone_regex = r'\+\d{1,3} \d{3} \d{3} \d{4}'
    matches = re.findall(phone_regex, number.get_attribute('outerHTML'))
    return matches[0]

def get_all_contacts(urls):
    contacts = []
    for url in urls:
        print(url)
        driver = webdriver.Firefox()
        driver.get(url)
        time.sleep(2)
        button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "tutorial__close"))
        )
        button.click()
        contact = get_contact(driver)
        contacts.append(contact)
        driver.close()
        driver.quit()
    return contacts

In [10]:
all_titles = []
all_prices = []
all_prices = []
all_subtitles = []
all_owners = []
all_text_previews = []
all_urls = []
all_contacts = []

for index in range(20, 41):

    driver = webdriver.Firefox()
    # close_button_class = "tutorial__close notes-tutorial__close"
    url = f"https://krisha.kz/arenda/kvartiry/almaty/?rent-period-switch=%2Farenda%2Fkvartiry&page={index}"
    driver.get(url)

    print("driver got url")
    
    time.sleep(random.random())
    button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "tutorial__close"))
        )
    button.click()

    print("driver clicked")

    titles = get_titles("a-card__title", driver)
    all_titles.append(titles)
    
    print("driver got titles")

    prices = get_prices("a-card__price", driver)
    all_prices.append(prices)
    
    print("driver got prices")

    subtitles = get_subtitles("a-card__subtitle", driver)
    all_subtitles.append(subtitles)

    print("driver got subtitles")
    
    owners = get_owners("a-card__owner", driver)
    all_owners.append(owners)
    
    print("driver got owners")

    text_previews = get_text_previews("a-card__text-preview", driver)
    all_text_previews.append(text_previews)

    print("driver got texts")
    
    urls = get_urls(driver)
    all_urls.append(urls)
    
    print("driver got urls")
    
    contacts = get_all_contacts(urls)
    all_contacts.append(contacts)
    
    driver.close()
    driver.quit()
    

driver got url
driver clicked
driver got titles
driver got prices
driver got subtitles
driver got owners
driver got texts
driver got urls
https://krisha.kz/a/show/691569735
https://krisha.kz/a/show/691503755
https://krisha.kz/a/show/691431647
https://krisha.kz/a/show/679525040
https://krisha.kz/a/show/687199015
https://krisha.kz/a/show/691009863
https://krisha.kz/a/show/682134446
https://krisha.kz/a/show/691419632
https://krisha.kz/a/show/690011605
https://krisha.kz/a/show/681985339
https://krisha.kz/a/show/691178154
https://krisha.kz/a/show/691119744
https://krisha.kz/a/show/691536685
https://krisha.kz/a/show/691568247
https://krisha.kz/a/show/686193057
https://krisha.kz/a/show/691568367
https://krisha.kz/a/show/691568123
https://krisha.kz/a/show/687657896
https://krisha.kz/a/show/677751433
https://krisha.kz/a/show/670198550
driver got url
driver clicked
driver got titles
driver got prices
driver got subtitles
driver got owners
driver got texts
driver got urls
https://krisha.kz/a/show

In [16]:
summary = 0
for i in all_titles:
    summary += len(i)
print(summary)

43


In [24]:
def get_contact_url(url):
    driver = webdriver.Firefox()
    driver.get(url)
    button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "tutorial__close"))
        )
    button.click()
    contact = driver.find_element(By.CLASS_NAME, "show-phones")
    contact.click()
    time.sleep(random.randint(1, 2))
    number = driver.find_element(By.CLASS_NAME, "offer__contacts-phones")
    phone_regex = r'\+\d{1,3} \d{3} \d{3} \d{4}'
    matches = re.findall(phone_regex, number.get_attribute('outerHTML'))
    return matches[0]

In [25]:
get_contact_url("https://krisha.kz/a/show/691508517")

['+7 700 249 1959']


In [28]:
for index in range(3, 5):

    driver = webdriver.Firefox()
    url = f"https://krisha.kz/arenda/kvartiry/almaty/?rent-period-switch=%2Farenda%2Fkvartiry&page={index}"
    driver.get(url)

    print("driver got url")
    
    time.sleep(random.random())
    button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "tutorial__close"))
        )
    button.click()

    print("driver clicked")

    titles = get_titles("a-card__title", driver)
    all_titles.append(titles)
    
    print("driver got titles")

    prices = get_prices("a-card__price", driver)
    all_prices.append(prices)
    
    print("driver got prices")

    subtitles = get_subtitles("a-card__subtitle", driver)
    all_subtitles.append(subtitles)

    print("driver got subtitles")
    
    owners = get_owners("a-card__owner", driver)
    all_owners.append(owners)
    
    print("driver got owners")

    text_previews = get_text_previews("a-card__text-preview", driver)
    all_text_previews.append(text_previews)

    print("driver got texts")
    
    urls = get_urls(driver)
    all_urls.append(urls)
    
    print("driver got urls")
    
    contacts = get_all_contacts(urls)
    all_contacts.append(contacts)
    
    driver.close()
    driver.quit()
    

driver got url
driver clicked
driver got titles
driver got prices
driver got subtitles
driver got owners
driver got texts
driver got urls
https://krisha.kz/a/show/691365737
https://krisha.kz/a/show/690316162
https://krisha.kz/a/show/680565603
https://krisha.kz/a/show/678979067
https://krisha.kz/a/show/681572502
https://krisha.kz/a/show/691405082
https://krisha.kz/a/show/691397629
https://krisha.kz/a/show/690244898
https://krisha.kz/a/show/690922783
https://krisha.kz/a/show/691205086
https://krisha.kz/a/show/691306506
https://krisha.kz/a/show/688537812
https://krisha.kz/a/show/687581868
https://krisha.kz/a/show/691321218
https://krisha.kz/a/show/670836899
https://krisha.kz/a/show/686929299
https://krisha.kz/a/show/691011055
https://krisha.kz/a/show/691524268
https://krisha.kz/a/show/690806256
https://krisha.kz/a/show/691236757
driver got url
driver clicked
driver got titles
driver got prices
driver got subtitles
driver got owners
driver got texts
driver got urls
https://krisha.kz/a/show

In [31]:
for index in range(5, 10):

    driver = webdriver.Firefox()
    # close_button_class = "tutorial__close notes-tutorial__close"
    url = f"https://krisha.kz/arenda/kvartiry/almaty/?rent-period-switch=%2Farenda%2Fkvartiry&page={index}"
    driver.get(url)

    print("driver got url")
    
    time.sleep(random.random())
    button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "tutorial__close"))
        )
    button.click()

    print("driver clicked")

    titles = get_titles("a-card__title", driver)
    all_titles.append(titles)
    
    print("driver got titles")

    prices = get_prices("a-card__price", driver)
    all_prices.append(prices)
    
    print("driver got prices")

    subtitles = get_subtitles("a-card__subtitle", driver)
    all_subtitles.append(subtitles)

    print("driver got subtitles")
    
    owners = get_owners("a-card__owner", driver)
    all_owners.append(owners)
    
    print("driver got owners")

    text_previews = get_text_previews("a-card__text-preview", driver)
    all_text_previews.append(text_previews)

    print("driver got texts")
    
    urls = get_urls(driver)
    all_urls.append(urls)
    
    print("driver got urls")
    
    contacts = get_all_contacts(urls)
    all_contacts.append(contacts)
    
    driver.close()
    driver.quit()

driver got url
driver clicked
driver got titles
driver got prices
driver got subtitles
driver got owners
driver got texts
driver got urls
https://krisha.kz/a/show/690498021
https://krisha.kz/a/show/686619987
https://krisha.kz/a/show/691321318
https://krisha.kz/a/show/690795785
https://krisha.kz/a/show/691086294
https://krisha.kz/a/show/689636166
https://krisha.kz/a/show/683676816
https://krisha.kz/a/show/690853991
https://krisha.kz/a/show/690868931
https://krisha.kz/a/show/689448248
https://krisha.kz/a/show/691527689
https://krisha.kz/a/show/691424707
https://krisha.kz/a/show/691527466
https://krisha.kz/a/show/691527173
https://krisha.kz/a/show/688441383
https://krisha.kz/a/show/691375247
https://krisha.kz/a/show/687674692
https://krisha.kz/a/show/691451105
https://krisha.kz/a/show/680424467
https://krisha.kz/a/show/690957222
driver got url
driver clicked
driver got titles
driver got prices
driver got subtitles
driver got owners
driver got texts
driver got urls
https://krisha.kz/a/show

In [32]:
summary = 0
for i in all_contacts:
    summary += len(i)
print(summary)

183


In [33]:
for index in range(10, 20):

    driver = webdriver.Firefox()
    # close_button_class = "tutorial__close notes-tutorial__close"
    url = f"https://krisha.kz/arenda/kvartiry/almaty/?rent-period-switch=%2Farenda%2Fkvartiry&page={index}"
    driver.get(url)

    print("driver got url")
    
    time.sleep(random.random())
    button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "tutorial__close"))
        )
    button.click()

    print("driver clicked")

    titles = get_titles("a-card__title", driver)
    all_titles.append(titles)
    
    print("driver got titles")

    prices = get_prices("a-card__price", driver)
    all_prices.append(prices)
    
    print("driver got prices")

    subtitles = get_subtitles("a-card__subtitle", driver)
    all_subtitles.append(subtitles)

    print("driver got subtitles")
    
    owners = get_owners("a-card__owner", driver)
    all_owners.append(owners)
    
    print("driver got owners")

    text_previews = get_text_previews("a-card__text-preview", driver)
    all_text_previews.append(text_previews)

    print("driver got texts")
    
    urls = get_urls(driver)
    all_urls.append(urls)
    
    print("driver got urls")
    
    contacts = get_all_contacts(urls)
    all_contacts.append(contacts)
    
    driver.close()
    driver.quit()

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event): operation timed out


driver got url
driver clicked
driver got titles
driver got prices
driver got subtitles
driver got owners
driver got texts
driver got urls
https://krisha.kz/a/show/691530757
https://krisha.kz/a/show/677090729
https://krisha.kz/a/show/683907061
https://krisha.kz/a/show/690383854
https://krisha.kz/a/show/689898831
https://krisha.kz/a/show/691376617
https://krisha.kz/a/show/691399626
https://krisha.kz/a/show/690334476
https://krisha.kz/a/show/688539389
https://krisha.kz/a/show/691530269
https://krisha.kz/a/show/681512152
https://krisha.kz/a/show/671149794
https://krisha.kz/a/show/691178154
https://krisha.kz/a/show/691074668
https://krisha.kz/a/show/691376330
https://krisha.kz/a/show/691249605
https://krisha.kz/a/show/670728441
https://krisha.kz/a/show/687101098
https://krisha.kz/a/show/689566897
https://krisha.kz/a/show/685951295
driver got url
driver clicked
driver got titles
driver got prices
driver got subtitles
driver got owners
driver got texts
driver got urls
https://krisha.kz/a/show

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event): operation timed out


https://krisha.kz/a/show/691504725
https://krisha.kz/a/show/691369256
https://krisha.kz/a/show/690092996
https://krisha.kz/a/show/691522083
https://krisha.kz/a/show/691114147
https://krisha.kz/a/show/691413252
https://krisha.kz/a/show/688922034
https://krisha.kz/a/show/683728533
https://krisha.kz/a/show/670631922
driver got url
driver clicked
driver got titles
driver got prices
driver got subtitles
driver got owners
driver got texts
driver got urls
https://krisha.kz/a/show/670631922
https://krisha.kz/a/show/691368555
https://krisha.kz/a/show/690853443
https://krisha.kz/a/show/691142934
https://krisha.kz/a/show/690680215
https://krisha.kz/a/show/671983676
https://krisha.kz/a/show/687630321
https://krisha.kz/a/show/691055818
https://krisha.kz/a/show/670579091
https://krisha.kz/a/show/680968716
https://krisha.kz/a/show/691065880
https://krisha.kz/a/show/691385611
https://krisha.kz/a/show/670876186
https://krisha.kz/a/show/691219160
https://krisha.kz/a/show/689008476
https://krisha.kz/a/sh

In [11]:
import sqlite3

def create_table():
    # Connect to SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('db_krisha.db')
    
    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()
    
    # Define a SQL query to create a table with 7 columns
    create_table_query = '''
        CREATE TABLE IF NOT EXISTS your_table_name (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            owner TEXT,
            contact TEXT,
            price REAL,
            subtitle TEXT,
            text_preview TEXT,
            title TEXT,
            url TEXT
        );
    '''
    
    # Execute the SQL query
    cursor.execute(create_table_query)
    
    # Commit changes and close the connection
    conn.commit()
    conn.close()

# Call the function to create the table
create_table()


In [13]:
def insert_values(all_owners, all_contacts, all_prices, all_subtitles, all_text_previews, all_titles, all_urls):
    # Connect to the SQLite database
    conn = sqlite3.connect('db_krisha.db')
    cursor = conn.cursor()

    # Iterate over the two-dimensional list and insert values into the database
    for i in range(len(all_prices)):
        for j in range(len(all_prices[i])):
            # Convert all values to string before insertion
            owner = str(all_owners[i][j])
            contact = str(all_contacts[i][j])
            price = str(all_prices[i][j])
            subtitle = str(all_subtitles[i][j])
            text_preview = str(all_text_previews[i][j])
            title = str(all_titles[i][j])
            url = str(all_urls[i][j])
            
            cursor.execute("INSERT INTO your_table_name (owner, contact, price, subtitle, text_preview, title, url) VALUES (?, ?, ?, ?, ?, ?, ?)",
                           (owner, contact, price, subtitle, text_preview, title, url))

    # Commit changes and close the connection
    conn.commit()
    conn.close()

# Call the function with your lists
insert_values(all_owners, all_contacts, all_prices, all_subtitles, all_text_previews, all_titles, all_urls)


In [128]:
con = sqlite3.connect("db_krisha.db")
df = pd.read_sql_query("SELECT * FROM your_table_name", con)
# df.head(10)

In [129]:
df.drop_duplicates(subset="url", inplace=True)

In [130]:
df["rooms"] = [i.split(",")[0][2:-1] for i in df.title]
df["apartment_size"] = [i.split(",")[1][2:-1] for i in df.title]
df["floor"] = [str(i.split(",")[-1]) + "этаж" if "этаж" in i else "-" for i in df.title]
df["rent"] = [j.split("этаж")[-1][1:-2] for j in [i.split(",")[-1] for i in df.title]]
# df.drop("title", axis=1, inplace=True)
df.rename(columns={"subtitle" : "district"}, inplace=True)
df.rename(columns={"owner" : "owner_type"}, inplace=True)
df.rename(columns={"text_preview" : "apartment_description"}, inplace=True)

In [131]:
df["rent"] = [i.split()[-1] if len(i.split()) > 1 else i for i in df["rent"]]
df["apartment_size"] = [ "".join(i.split()[:-1]) if "помесячно" in i else i for i in df.apartment_size]

In [136]:
# for i in df.title:
#     if "этаж" not in i:
#         print("=====================")
#         print(i.split(",")[-1])
#         print("---------------------")
#         print(i)
#         print("=====================")

In [98]:
def has_district(t):
    if "р-н" in t.lower() or "район" in t.lower():
        return True
    return False

def get_district(t):
    if has_district(t):
        for i in t.split(","):
            if has_district(i):
                return i
    else:
        return "-"
    
def get_address(t):
    counter = 0
    if has_district(t.lower()):
        if len(t.split(",")[1:]) == 1:
            return str(t.split(",")[-1])
        else:
            address = ""
            counter += 1
            for i in t.split(",")[1:]:
                address += i
                counter += 1
            return address
    else:
        return t

In [99]:
df["Districts"] = df["district"].apply(get_district)
df["address"] = df["district"].apply(get_address)

In [100]:
df.drop("district", axis=1, inplace=True)

In [101]:
df.columns

Index(['id', 'owner_type', 'contact', 'price', 'apartment_description', 'url',
       'rooms', 'apartment_size', 'floor', 'rent', 'Districts', 'address'],
      dtype='object')

In [102]:
df.drop(columns=df.columns[0], axis=1, inplace=True)
df.head(3)

,owner_type,contact,price,apartment_description,url,rooms,apartment_size,floor,rent,Districts,address
0,Компания,+7 747 975 2075,250000.0,"жил. площадь 20 м², кухня 12 м², меблирована п...",https://krisha.kz/a/show/691390570,1-комнатная квартира,43 м²,5/10 этаж,помесячно,Медеуский р-н,Нусупбекова — Янушкевича
1,Специалист,+7 778 778 2165,260000.0,"меблирована полностью, В отличном состояние!",https://krisha.kz/a/show/690991367,2-комнатная квартира,66 м²,7/9 этаж,помесячно,Алатауский р-н,мкр Аккент 4
2,Специалист,+7 707 322 2190,250000.0,"жил. площадь 45 м², кухня 14 м², меблирована ч...",https://krisha.kz/a/show/690986537,3-комнатная квартира,74 м²,7/9 этаж,помесячно,Ауэзовский р-н,мкр Мамыр-4 302


In [103]:
df.groupby("contact").__len__()

469

In [104]:
df.__len__()

588

In [105]:
df_without_duplicates = df.drop_duplicates(subset="url")

In [106]:
df_without_duplicates.__len__()

588

In [14]:
# def drop_duplicates(t:pd.DataFrame):
    # t.drop_duplicates(subset="url", inplace=True)
    # t.drop_duplicates(subset=["apartment_description", "rooms", "apartment_size", "rent"], inplace=True)
# df_without_duplicates = df_without_duplicates.drop_duplicates(subset="url")
# df_without_duplicates = df_without_duplicates.drop_duplicates(subset=["rooms", "apartment_size", "rent"], keep="last")

In [15]:
df_without_duplicates.__len__()

196

In [107]:
df_without_duplicates.to_csv("krisha.csv")